In [27]:
import requests
import logging
import base64
import pandas as pd
import json

class CoreDataService:
    """
    Clase para conectar y recuperar datos de SAP CDS views.
    
    Attributes:
        base_url (str): URL base para la conexión con la API.
        batch_mode (bool): Indica si las consultas deben ser procesadas en lotes.
        batch_size (int): Tamaño de cada lote si batch_mode es True.
    """
    
    def __init__(self, base_url, batch_mode=True, batch_size=150):
        """
        Inicializa la instancia con la configuración de conexión necesaria.
        
        Args:
            base_url (str): URL base para la conexión con la API.
            batch_mode (bool): Habilita o deshabilita la consulta en lotes.
            batch_size (int): Define el tamaño de cada lote para la consulta.
        """
        self.base_url = base_url
        self.session = requests.Session()
        self.batch_mode = batch_mode
        self.batch_size = batch_size
        logging.info(f"CoreDataService initialized with base URL: {base_url}")

    def query_cds(self, materials, field, from_cds, column_name):
        """
        Consulta datos según la configuración de batch_mode y maneja errores por lote o por material.
        
        Args:
            materials (list): Lista de materiales para consulta.
            field (str): Campo seleccionado para el query.
            from_cds (str): Nombre de la vista CDS desde donde se consulta.
            column_name (str): Nombre de la columna para el filtro.
        
        Returns:
            tuple: Un DataFrame con todos los resultados concatenados y un diccionario de errores.
        """
        results = []
        errors = {}
        if self.batch_mode:
            for i in range(0, len(materials), self.batch_size):
                batch = materials[i:i + self.batch_size]
                result, batch_errors = self.process_batch(batch, field, from_cds, column_name)
                if result is not None:
                    results.append(result)
                if batch_errors:
                    errors.update(batch_errors)
        else:
            for material in materials:
                try:
                    result = self.process_individual(material, field, from_cds, column_name)
                    if result is not None and not result.empty:
                        results.append(result)
                except Exception as e:
                    logging.error(f"Error processing material {material}: {str(e)}")
                    errors[material] = str(e)

        return pd.concat(results, ignore_index=True) if results else pd.DataFrame(), errors

    def process_batch(self, batch, field, from_cds, column_name):
        """
        Procesa un lote de materiales y devuelve los resultados y errores.
        """
        json_query = self.build_query(batch, field, from_cds, column_name)
        try:
            # Asumiendo que transform_json_input_to_df solo devuelve un DataFrame y no una tupla
            result = self.transform_json_input_to_df(json_query)
            return result, {}  # Devuelve un diccionario vacío de errores si todo sale bien
        except Exception as e:
            logging.error(f"Error processing batch: {str(e)}")
            # Devuelve un DataFrame vacío y un diccionario con los errores correspondientes al lote
            return None, {mat: str(e) for mat in batch}

    def process_individual(self, material, field, from_cds, column_name):
        """
        Procesa un único material y devuelve el resultado.
        """
        json_query = self.build_query([material], field, from_cds, column_name)
        return self.transform_json_input_to_df(json_query)

    def transform_json_input_to_df(self, json_data):
        """
        Envía un query JSON a la API y retorna un DataFrame.
        """
        base64_data = base64.b64encode(json.dumps(json_data).encode()).decode()
        url = f'{self.base_url}{base64_data}'
        response = self.session.get(url)
        response.raise_for_status()
        return pd.DataFrame(response.json())

    def build_query(self, materials, column_name, start_date, end_date, table='ZMMT010ZMMT0072', field='*'):
        """
        Construye un diccionario con la cláusula WHERE para filtrar por una lista de materiales y un rango de fechas.

        Args:
            materials (list of str): lista de valores para filtrar en column_name.
            column_name (str): nombre de la columna para aplicar el filtro de materiales.
            start_date (str or datetime): fecha de inicio en formato 'YYYYMMDD' o datetime.
            end_date (str or datetime): fecha de fin en formato 'YYYYMMDD' o datetime.
            table (str): nombre de la tabla para la fuente de datos.
            field (str): campos a seleccionar.

        Returns:
            dict: diccionario con llaves 'FIELD', 'FROM', 'WHERE'.
        """
        # Convertir fechas si vienen como datetime
        try:
            import datetime
            if isinstance(start_date, datetime.date):
                start_date = start_date.strftime('%Y%m%d')
            if isinstance(end_date, datetime.date):
                end_date = end_date.strftime('%Y%m%d')
        except ImportError:
            pass

        # Construir filtro de fechas
        date_filter = f"ERDAT BETWEEN '{start_date}' AND '{end_date}'"
        # Construir filtro de materiales unido por OR
        material_filter = " or ".join([f"{column_name} eq '{m}'" for m in materials])
        # Combinar todo en la cláusula WHERE
        where_clause = f"{date_filter} AND ({material_filter})"

        # Armar y retornar el diccionario final
        return {
            'FIELD': field,
            'FROM': table,
            'WHERE': where_clause
        }
        


In [ ]:
service = CoreDataService("http://sapapp01.prebel.com.co:8000/services/zws_gl_0001?SELECT=", batch_mode=True)
data, errors = service.query_cds(["*"], "*", "ZMMT010ZMMT0072", "MAKTX")

In [ ]:
service = CoreDataService("http://sapapp01.prebel.com.co:8000/services/zws_gl_0001?SELECT=", batch_mode=True)
data, errors = service.build_query(["*"], "MAKTX", "20250331", "20250430")

In [18]:
data

,MANDT,ERDAT,MATNR,MAKTX,MEINS,MAABC,MTART,MARCA,ZNEGOC,LABST,...,METASKU,METAABC,METAMARCA,EXCEMES,STOCKID,EXCESUN,EXCESVR,STATUS,COBERTURA,WAERS
0,400,2025-04-30,PB0045272,LAB VITU ACE ALMEND FPS 10 SANDIA,ST,O,FERT,017,RETAIL,8.000,...,1.02,1.0,5.0,3,2.2,5.8,189.20,V,2.Entre 90 y 180 dias,COP
1,400,2025-04-30,PB0045764,NCRESPADOR UBU PESTAÑAS,ST,O,ZMER,038,RETAIL,6.000,...,0.00,1.0,0.0,0,0.0,6.0,263.46,V,1.Menor de 90 dias,COP
2,400,2025-04-30,PB0045799,ENCRESPADOR QVS PESTAÑAS,ST,O,ZMER,036,RETAIL,106.000,...,0.00,1.0,0.0,1,0.0,106.0,4175.34,V,1.Menor de 90 dias,COP
3,400,2025-04-30,PB0046633,ST VITU BODY SPLASH FELICITY 240 ML,ST,O,HALB,017,RETAIL,52.000,...,0.00,1.0,5.0,0,0.0,52.0,2181.92,V,1.Menor de 90 dias,COP
4,400,2025-04-30,PB0046675,WIDELASH,KG,A,ROH,017,RETAIL,1.912,...,0.00,1.0,5.0,1,1.0,0.9,11152.92,V,1.Menor de 90 dias,COP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19457,400,2025-03-31,PB0098669,BCKLB AC AVOC OIL BODYLOT 1L_E,ST,,VERP,015,FPT,27968.000,...,0.00,0.0,3.0,0,0.0,27968.0,33561.60,V,1.Menor de 90 dias,COP
19458,400,2025-03-31,PB0098689,AC ACCOLADE ANTI-AGEING MASK 7,ST,,VERP,015,FPT,17709.000,...,0.00,0.0,3.0,10,4160.0,13549.0,132102.75,R,5.Mayor o igual a 360 dias,COP
19459,400,2025-03-31,PB0098721,SOB DEO PIES SNA AR GEN CIL PP NE,ST,A,VERP,001,RETAIL,58448.000,...,0.00,1.0,3.1,2,17967.4,40480.6,131966.76,V,2.Entre 90 y 180 dias,COP
19460,400,2025-03-31,PB0098838,PAL.RUB.ILUM.CATR. CHEEK AFF.TN030 10G,ST,O,ZMER,057,RETAIL,1.000,...,0.00,1.0,5.6,0,3.9,0.0,0.00,V,1.Menor de 90 dias,COP


In [ ]:
data2 = data.sort_values(by='ERDAT', ascending=True)
data2